In [38]:
import wandb
import pandas as pd

In [39]:
api = wandb.api()

TypeError: 'Api' object is not callable

In [86]:
TASKS = ["PredictFluPos",
        "PredictSurveyCol-have_flu > 0",
        "EarlyDetection",
        "PredictTrigger",
        "PredictSurveyCol-(symptom_severity__q_fatigue_2 > 0) | (symptom_severity__q_fatigue_3 > 0)"]

MODELS = [
    "CNNToTransformerEncoder",
    "XGBoost"
]

In [87]:
# Change oreilly-class/cifar to <entity/project-name>
runs = api.runs("mikeamerrill/flu")
summary_list = [] 
config_list = [] 
name_list = [] 
for run in runs: 
    # run.summary are the output key/values like accuracy.  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.  We remove special values that start with _.
    config_list.append({k:v for k,v in run.config.items() if not k.startswith('_')}) 

    # run.name is the name of the run.
    name_list.append(run.name)       

summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 

summary_df["task"] = summary_df["task"].fillna(config_df["task"])
summary_df["eval/roc_auc"] = summary_df["eval/roc_auc"].fillna(summary_df["eval-auc"])

del config_df["task"]

name_df = pd.DataFrame({'name': name_list}) 
all_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [88]:
results = all_df[all_df["model"].isin(MODELS) & all_df["task"].isin(TASKS)]

In [89]:
results.groupby(["task","model"])["eval/roc_auc"].max()

task                                                                                        model                  
EarlyDetection                                                                              CNNToTransformerEncoder    0.577686
                                                                                            XGBoost                    0.784357
PredictFluPos                                                                               CNNToTransformerEncoder    0.948252
                                                                                            XGBoost                    0.744008
PredictSurveyCol-(symptom_severity__q_fatigue_2 > 0) | (symptom_severity__q_fatigue_3 > 0)  CNNToTransformerEncoder    0.674691
                                                                                            XGBoost                    0.717857
PredictSurveyCol-have_flu > 0                                                               CNNToTransformerEncoder 

In [82]:
all_df[(all_df["name"].str.contains("282"))]["eval/roc_auc"]

166   NaN
Name: eval/roc_auc, dtype: float64

In [72]:
all_df["name"]

0          flowing-fog-477
1       feasible-disco-476
2        glowing-brook-475
3      hardy-resonance-474
4         woven-dragon-473
              ...         
412        radiant-water-6
413           rural-fire-5
414         blooming-fog-4
415         scarlet-tree-3
416       polished-disco-1
Name: name, Length: 417, dtype: object